## Import

In [2]:
from ilthermo.models import *
ions = session.query(Ion)
mols = session.query(Molecule)
import sys
sys.path.append('../ms-tools')

## Canonicalize SMILES

In [2]:
for ion in ions.filter(Ion.ignored==False):
    m = pybel.readstring('smi', ion.smiles)
    ion.smiles = m.write('can').strip()
session.commit()

## Check molcular formula

In [3]:
for mol in mols:
    if not mol.cation.ignored and not mol.anion.ignored:
        smiles = mol.cation.smiles + '.' + mol.anion.smiles
        m = pybel.readstring('smi', smiles)
        if mol.formula != m.formula:
            print(mol, mol.formula, m.formula, mol.cation_id, mol.anion_id)

## Check number of papers for each ion

In [ ]:
for ion in ions.filter(Ion.ignored==False):
    ps = set()
    for mol in ion.molecules:
        for data in mol.datas:
            ps.add(data.paper)
    print(ion, len(ps))
    ion.n_paper = len(ps)
session.commit()

## Check duplicated ions

In [58]:
smiles_ion = dict()
for ion in ions.filter(Ion.ignored==False).order_by(Ion.n_paper.desc()):
    if ion.smiles not in smiles_ion.keys():
        smiles_ion[ion.smiles] = ion
    else:
        print(ion, smiles_ion[ion.smiles])
        ion.duplicate = smiles_ion[ion.smiles].id
session.commit()

<Ion -1: 22 bis(trifluoromethylsulfonyl)imide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 87 bis((trifluoromethyl)sulfonyl)amide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion +1: 374 1-methyl-3-octylimidazolium> <Ion +1: 265 1-octyl-3-methylimidazolium>
<Ion -1: 310 bis(trifluoromethylsulfonyl)amide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 93 1,1,1-trifluoro-N-[(trifluoromethyl)sulfonyl]methanesulfonamide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 740 methylsulfate> <Ion -1: 840 methyl sulfate>
<Ion +1: 10 1-ethyl-3-methyl-1H-imidazolium> <Ion +1: 105 1-ethyl-3-methylimidazolium>
<Ion -1: 97 bis(trifluoromethanesulfonyl)imide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 98 ethylsulfate> <Ion -1: 841 ethyl sulfate>
<Ion -1: 129 bis((trifluoromethyl)sulfonyl)imide> <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
<Ion -1: 34 tricyanomethane> <Ion -1: 101 tricyanomethanide>
<Ion +1: 445 tetradecyl(trihexyl)phosphonium

## Check number of papers for unique ion

In [9]:
for ion in ions.filter(Ion.ignored==False).order_by(Ion.n_paper.desc()):
    if ion.duplicate != None:
        ion.times = 0
    else:
        ion.times = ion.n_paper
        for eqi in ions.filter(Ion.duplicate==ion.id):
            ion.times += eqi.n_paper
    print(ion.times, ion)
session.commit()

483 <Ion +1: 2 1-butyl-3-methylimidazolium>
414 <Ion +1: 105 1-ethyl-3-methylimidazolium>
299 <Ion -1: 24 tetrafluoroborate>
663 <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
228 <Ion +1: 264 1-hexyl-3-methylimidazolium>
0 <Ion -1: 22 bis(trifluoromethylsulfonyl)imide>
159 <Ion -1: 20 hexafluorophosphate>
108 <Ion -1: 30 chloride>
127 <Ion -1: 841 ethyl sulfate>
113 <Ion -1: 33 trifluoromethanesulfonate>
107 <Ion -1: 29 dicyanamide>
88 <Ion +1: 14 1-butyl-1-methylpyrrolidinium>
0 <Ion -1: 87 bis((trifluoromethyl)sulfonyl)amide>
170 <Ion +1: 265 1-octyl-3-methylimidazolium>
79 <Ion -1: 27 bromide>
0 <Ion +1: 374 1-methyl-3-octylimidazolium>
64 <Ion -1: 28 acetate>
0 <Ion -1: 310 bis(trifluoromethylsulfonyl)amide>
93 <Ion -1: 840 methyl sulfate>
49 <Ion -1: 38 thiocyanate>
51 <Ion +1: 497 1,3-dimethylimidazolium>
46 <Ion +1: 12 1-butylpyridinium>
46 <Ion +1: 280 1-butyl-3-methylpyridinium>
0 <Ion -1: 93 1,1,1-trifluoro-N-[(trifluoromethyl)sulfonyl]methanesulfonamide>
0 <Ion -1: 740 me

1 <Ion +1: 173 N,N-diethylpropan-1-aminium>
1 <Ion +1: 174 N-methyl-N-propylpropan-1-aminium>
1 <Ion +1: 175 N,N-dimethylpropan-1-aminium>
1 <Ion +1: 176 (3-carboxypropyl)trimethylammonium>
2 <Ion -1: 177 docusate>
0 <Ion +1: 178 N-(2-hydroxyethyl)-N,N-dimethyl-1-propanaminium>
1 <Ion -1: 180 heptadecafluorooctanesulfonate>
3 <Ion +1: 181 ethyldimethyl-(2-methoxyethyl)ammonium>
0 <Ion +1: 191 N,N,N-trioctyl-N-methylammonium>
0 <Ion +1: 192 N,N,N-triheptylheptan-1-aminium>
2 <Ion +1: 193 N,N,N-trihexylheptan-1-aminium>
2 <Ion +1: 194 butyltris(3-methylbutyl)ammonium>
0 <Ion +1: 197 N-heptyl-N,N,N-trihexylammonium>
0 <Ion +1: 198 N-ethyl-2-hydroxyethyl-N,N-dimethylammonium>
1 <Ion +1: 199 (2-hydroxyethyl)-dimethylundecyloxymethylammonium>
1 <Ion +1: 200 (2-methyloyoxyethyl)dimethylpentyloxyammonium>
1 <Ion +1: 202 (2-decanoyloxyethyl)dimethylpentyloxymethylammonium>
0 <Ion +1: 207 hexadecyltrimethylammonium>
1 <Ion +1: 208 ethyltrimethylammonium>
1 <Ion +1: 210 2-methylpropylammonium>
1 

1 <Ion +1: 919 1-butyl-3-(4-sulfobutyl)-imidazolium>
1 <Ion +1: 921 1-methyl-3-(3-sulfopropyl)-imidazolium>
1 <Ion +1: 922 1-nonylammonium>
1 <Ion +1: 923 1-octanaminium>
1 <Ion -1: 925 butylphosphonate>
1 <Ion -1: 934 octoate>
2 <Ion +1: 937 cyclohexanamine>
1 <Ion -1: 938 heptadecanoic acid>
0 <Ion -1: 939 octadecanoic acid>
0 <Ion -1: 940 dodecanoic acid>
0 <Ion +1: 941 cyclohexylamine>
1 <Ion -1: 942 tridecanoic acid>
0 <Ion -1: 943 hexadecanoic acid>
0 <Ion +1: 836 4-(1-hexadecylheptadecyl)-1-methyl-pyridinium>
0 <Ion +1: 846 N,N,N-trihexyl-1-heptanaminium>
0 <Ion +1: 847 N-heptyl-N,N-dihexyl-1-heptanaminium>
0 <Ion +1: 856 N-ethyl-N,N-dimethyl-2-methoxyethylammonium>
0 <Ion +1: 884 N-dodecyl-N,N-dimethylbenzenemethanaminium>


## Show number of heavy atoms

In [39]:
for ion in ions.filter(Ion.selected==True):
    print(ion.n_heavy_atoms, ion)

10 <Ion +1: 2 1-butyl-3-methylimidazolium>
9 <Ion +1: 5 1-methyl-1-propylpyrrolidinium>
9 <Ion -1: 6 bis(fluorosulfonyl)amide>
12 <Ion +1: 8 1-hexylpyridinium>
8 <Ion +1: 11 1-ethylpyridinium>
10 <Ion +1: 12 1-butylpyridinium>
10 <Ion +1: 14 1-butyl-1-methylpyrrolidinium>
7 <Ion -1: 20 hexafluorophosphate>
7 <Ion -1: 21 trifluoroacetate>
4 <Ion -1: 23 nitrate>
5 <Ion -1: 24 tetrafluoroborate>
4 <Ion -1: 28 acetate>
5 <Ion -1: 29 dicyanamide>
1 <Ion -1: 30 chloride>
8 <Ion -1: 33 trifluoromethanesulfonate>
7 <Ion -1: 36 dimethylphosphate>
3 <Ion -1: 38 thiocyanate>
9 <Ion -1: 41 tetracyanoborate>
5 <Ion -1: 42 perchlorate>
5 <Ion -1: 54 methanesulfonate>
3 <Ion -1: 57 formate>
15 <Ion -1: 81 bis[(trifluoromethyl)sulfonyl]imide>
7 <Ion -1: 101 tricyanomethanide>
8 <Ion +1: 105 1-ethyl-3-methylimidazolium>
5 <Ion -1: 106 propionate>
8 <Ion +1: 126 butyltrimethylammonium>
6 <Ion +1: 172 diethylmethylammonium>
12 <Ion +1: 264 1-hexyl-3-methylimidazolium>
17 <Ion +1: 296 tetrabutylphosphoniu

## Select molecules

In [46]:
for mol in mols:
    mol.selected = False
    cation = mol.cation
    anion = mol.anion
    if cation.duplicate != None:
        cation = ions.get(cation.duplicate)
    if anion.duplicate != None:
        anion = ions.get(anion.duplicate)
    if cation.selected and anion.selected:
        mol.selected = True
        print(mol.cation.category, mol.anion.category, mol)
session.commit()

cIm aSI <Molecule: 1 1-butyl-3-methylimidazolium bis(fluorosulfonyl)imide>
cPyrr aSI <Molecule: 3 1-methyl-1-propylpyrrolidinium bis(fluorosulfonyl)amide>
cPy aSI <Molecule: 5 1-hexylpyridinium bis(fluorosulfonyl)amide>
cIm aSI <Molecule: 7 1-ethyl-3-methyl-1H-imidazolium bis(fluorosulfonyl)amide>
cPy aSI <Molecule: 8 1-ethylpyridinium bis(fluorosulfonyl)amide>
cPy aSI <Molecule: 9 1-butylpyridinium bis(fluorosulfonyl)amide>
cPyrr aSI <Molecule: 11 1-butyl-1-methylpyrrolidinium bis(fluorosulfonyl)amide>
cIm aXP6 <Molecule: 17 1-butyl-3-methylimidazolium hexafluorophosphate>
cIm aCO2 <Molecule: 18 1-butyl-3-methylimidazolium trifluoroacetate>
cIm aSI <Molecule: 19 1-butyl-3-methylimidazolium bis(trifluoromethylsulfonyl)imide>
cIm aNO3 <Molecule: 20 1-butyl-3-methylimidazolium nitrate>
cIm aXB4 <Molecule: 21 1-butyl-3-methylimidazolium tetrafluoroborate>
cIm aCO2 <Molecule: 25 1-butyl-3-methylimidazolium acetate>
cIm aNC[N] <Molecule: 26 1-butyl-3-methylimidazolium dicyanamide>
cIm a0Cl 

## Fit density and viscosity for selected molecuels

In [36]:
mols.update({'fit':None})
session.commit()
#for mol in mols.filter(Molecule.selected==True):
#    print(mol)
#    try:
#        score = mol.fit_density()
#        print(score)
#    except:
#        pass
#    print(mol.fit)

## Misc

In [44]:
ion = ions.get(185)
ion.molecules.all()

[<Molecule: 210 triethylammonium acetate>,
 <Molecule: 1289 triethylammonium hydrogen sulfate>,
 <Molecule: 1304 triethylammonium dihydrogen phosphate>]

In [37]:
mol = mols.get(1)
print(mol.cation)
print(mol.anion)


<Ion +1: 447 1,4-dimethylpyridinium>
<Ion -1: 442 4-methylbenzenesulfonate>


In [85]:
mol = mols.get(1)
vis = session.query(Property).filter(Property.name=='Viscosity').first()
datas = mol.datas.filter(Data.property==vis)
print(datas.all())

[<Data: Viscosity: 273.1 101.0 0.098800>, <Data: Viscosity: 278.1 101.0 0.076500>, <Data: Viscosity: 283.1 101.0 0.060400>, <Data: Viscosity: 288.1 101.0 0.048600>, <Data: Viscosity: 293.1 101.0 0.039700>, <Data: Viscosity: 298.1 101.0 0.032900>, <Data: Viscosity: 303.1 101.0 0.027600>, <Data: Viscosity: 313.1 101.0 0.020220>, <Data: Viscosity: 323.1 101.0 0.015350>, <Data: Viscosity: 333.1 101.0 0.012010>, <Data: Viscosity: 343.1 101.0 0.009650>, <Data: Viscosity: 353.1 101.0 0.007920>, <Data: Viscosity: 363.1 101.0 0.006630>]
